Ensure that our database is ready

In [ ]:
%%bash
if [[ -d project-tycho-utilities ]];
then
  cd project-tycho-utilities/
  git pull
else
  git clone https://github.com/lgautier/project-tycho-utilities.git
  cd project-tycho-utilities/
fi
DBNAME=../tycho.db make all

---

In [ ]:
%load_ext rpy2.ipython

R packages can be imported as is they were Python packages:

<!-- label:rpy2_importr -->

In [ ]:
from rpy2.robjects.packages import importr

stats = importr('stats')


<!-- label:rpy2_rnorm -->

In [ ]:

tuple(stats.rnorm(5))


---

<!-- label:dplyr_table -->
dplyr is not trying to map objects. It is focusing on databases
as sources of tables.

In [ ]:
from rpy2.robjects.lib import dplyr

dbfilename = "tycho.db"
datasrc  = dplyr.src_sqlite(dbfilename)
location_tbl = datasrc.get_table("location")

<!-- label:dplyr_query -->
The table can be queried using the dplyr interface.

In [ ]:
res =  (location_tbl
        .filter('state %like% "M%"')
        .group_by('state')
        .count('state')
        .arrange('desc(n)'))

print(res)


Strings are snippets of R code for dplyr.

R can be considered a domain-specific language (DSL) in the Python code.

---

<!-- label:dplyr_advanced -->

Let's implement our complex SQL query from early with dplyr.

In [ ]:
casecount_tbl = datasrc.get_table("casecount")

## Count diseases
disease_per_city = (casecount_tbl
                    .group_by('location_id')
                    .summarize(n='count(distinct(disease_id))'))
## Filter cities
high_disease = (disease_per_city
                .filter('n > 5'))
## Join location
inner_join = dplyr.inner_join
join_location = inner_join((location_tbl
                            .mutate(location_id="id")),
                           high_disease,
                           by="location_id")
res = (dplyr.DataFrame(join_location)
       .group_by('state')
       .summarize(n='count(city)')
       .arrange('desc(n)')
       .collect())


---

<!-- label:ggplot2_figure -->
The R package ggplot2 can also be used.

In [ ]:
from rpy2.robjects import r, globalenv
import rpy2.robjects.lib.ggplot2 as gg

p = (gg.ggplot(res.head(20)) +
     gg.geom_bar(
       gg.aes_string(x='factor(state, levels=as.character(state))', 
                     y='n'),
                 stat='identity') +
     gg.scale_x_discrete("State") +
     gg.scale_y_sqrt("# cities w/ >5 diseases"))

<!-- label:ggplot2_plot -->
<!-- config:split-output -->
Sending the resulting figure to a jupyter notebook output.

In [ ]:
from rpy2.ipython.ggplot import image_png
from IPython.display import display

display(image_png(p))

---

<!-- label:ggplot2_plot_map -->
<!-- config:split-output -->

In [ ]:
from rpy2.robjects.packages import importr
from rpy2.robjects import baseenv

maps = importr('maps')
# R dataset with map information in regions 
states = dplyr.DataFrame(gg.map_data('state'))
# R dataset mapping states and regions
state_abb = (dplyr.DataFrame({'state': baseenv.get('state.abb'),
                              'region': baseenv.get('state.name')})
             .mutate(region = 'tolower(region)'))
# Join both
states_map = dplyr.inner_join(states, state_abb, by="region")
dataf_plot = (dplyr.inner_join(states_map, res, by="state")
	      .arrange('order'))

p = (gg.ggplot(dataf_plot) +
     gg.geom_polygon(gg.aes_string(x='long', y='lat',
                                   group='group', fill='n')) +
     gg.scale_fill_continuous(trans="sqrt") +
     gg.coord_map("albers",  at0 = 45.5, lat1 = 29.5) +
     gg.theme_gray(base_size = 12))
display(image_png(p))

---

<!-- label:ggplot2_plot_map_gilbert -->
<!-- config:split-output -->

In [ ]:
p = (gg.ggplot(dataf_plot) +
     gg.geom_polygon(gg.aes_string(x='long', y='lat',
                                   group='group', fill='n')) +
     gg.scale_fill_continuous(trans="sqrt") +
     gg.theme_gray(base_size = 12))
     
display(
    image_png(p +
              gg.coord_map("gilbert")))

---

<!-- label:ggplot2_plot_map_azequalarea -->
<!-- config:split-output -->

In [ ]:
from rpy2.robjects.vectors import FloatVector
# Centered on New York
display(
    image_png(p +
              gg.geom_polygon(gg.aes_string(x='long', y='lat',
                                            group='group'),
                              alpha = 0.2,
                              data=gg.map_data("world")) +
              gg.coord_map("azequalarea",
                           orientation = FloatVector([41, -74, 0])) +
              gg.labs(title = "Centered on New York")))

---
<!-- label:ggplot2_plot_pneumonia_prepare_1_2 -->

In [ ]:
sql_disease = """
SELECT date_from, count, event, city
FROM casecount
INNER JOIN disease
ON casecount.disease_id=disease.id
INNER JOIN location
ON casecount.location_id=location.id
WHERE disease.name='%s'
AND state='%s'
AND city IS NOT NULL
"""
sql = sql_disease % ('PNEUMONIA', 'MA')

<!-- label:ggplot2_plot_pneumonia_prepare_2_2 -->

In [ ]:
robj = dplyr.tbl(datasrc, dplyr.dplyr.sql(sql))
dataf = (dplyr.DataFrame(robj).collect()
         .mutate(date='as.POSIXct(strptime(date_from, format="%Y-%m-%d"))')
         .mutate(month = 'format(date, "%m")',
                 year = 'format(date, "%Y")'))
# sum per month
dataf_plot = (dataf
              .group_by('city', 'event', 'month','year')
              .summarize(count='sum(count)'))
# 
yearmonth_to_date = """
as.POSIXct(
    strptime(
        paste(year, month, "15", sep="-"),
        format="%Y-%m-%d")
    )
"""

dataf_plot = dataf_plot.mutate(date=yearmonth_to_date)

Initial plot:

In [ ]:
from rpy2.robjects import Formula
p = (gg.ggplot(dataf_plot) +
     gg.geom_line(gg.aes_string(x='date', y='count',
                                group='city')) +
     gg.scale_y_sqrt() +
     gg.facet_wrap(Formula('~ event')))
display(image_png(p))

Color Boston vs Other

In [ ]:
p = (gg.ggplot(dataf_plot
               .mutate(city_label='ifelse(city=="BOSTON", "Boston", "Other")')) +
     gg.geom_line(gg.aes_string(x='date', y='count',
                                group='city', color='city_label')) +
     gg.scale_y_sqrt() +
     gg.facet_wrap(Formula('~ event')))
display(image_png(p))

Something is strange before approx. 1925. Let's focus on the most recent
data.

In [ ]:
p = (gg.ggplot(dataf_plot
               .mutate(city_label='ifelse(city=="BOSTON", "Boston", "Other")')
               .filter('year > 1925')) +
     gg.geom_line(gg.aes_string(x='date', y='count',
                                group='city', color='city_label')) +
     gg.scale_y_sqrt() +
     gg.facet_wrap(Formula('~ event')))
display(image_png(p))

Data for relatively few city. We can color them individually.

In [ ]:
p = (gg.ggplot(dataf_plot
               .filter('year > 1925')) +
     gg.geom_line(gg.aes_string(x='date', y='count',
                                group='city', color='city')) +
     gg.scale_y_sqrt() +
     gg.facet_wrap(Formula('~ event')))
display(image_png(p))

Decreasing for Boston, increasing for Worcester.

In [ ]:
p = (gg.ggplot(dataf_plot
               .filter('year > 1925')) +
     gg.geom_line(gg.aes_string(x='date', y='count',
                                group='city', color='city'),
                  alpha=0.4) +
     gg.geom_smooth(gg.aes_string(x='date', y='count',
                                group='city', color='city')) +
     gg.scale_y_sqrt() +
     gg.facet_wrap(Formula('~ event')))
display(image_png(p))

<!-- label:ggplot2_plot_pneumonia -->
<!-- config:split-output -->

In [ ]:
p = (gg.ggplot(dataf_plot
               .filter('year > 1925')) +
     gg.geom_line(
       gg.aes_string(x='month', y='count',
                     group='paste(year, city)', color='city')) +
     gg.facet_grid('city~.', scales="free_y") +
     gg.scale_y_sqrt() +
     gg.facet_wrap(Formula('~ event')))
display(image_png(p))

Which years correspond to largest number of cases in Boston ?

In [ ]:
set((dataf_plot
     .filter('count>200', 'year>1925')
     .rx2('year')))

Corresponds to largest number of cases in Fall Rivers.
Springfield and Worcester have different bad years.

In [ ]:
p = (gg.ggplot(dataf_plot
               .filter('year > 1925')) +
     gg.geom_line(gg.aes_string(x='month', y='count', 
                                group='year', 
                                color='year %in% c(1926,1929,1931,1933,1937)')) +
     gg.facet_grid(Formula('city ~ event'), scales="free_y"))
display(image_png(p))

---

Function to plot monthly aggregates.

<!-- label:function_make_ggplot -->

In [ ]:
def myplot(disease, state):
    sql = sql_disease % (disease, state)
    dataf = (dplyr.DataFrame(dplyr.tbl(datasrc, dplyr.dplyr.sql(sql))).collect()
             .mutate(date='as.POSIXct(strptime(date_from, format="%Y-%m-%d"))')
             .mutate(month = 'format(date, "%m")',
                     year = 'format(date, "%Y")'))

    dataf_plot = (dataf
                  .group_by('city', 'event', 'month','year')
                  .summarize(count='sum(count)'))
    
    dataf_plot = dataf_plot.mutate(date=yearmonth_to_date)
    p = (gg.ggplot(dataf_plot
                    .filter('year > 1925')) +
         gg.geom_line(gg.aes_string(x='month', y='count+1',
                                    group='year', color='city')) +
         gg.facet_grid(Formula('city ~ event'), scales="free_y") +
         gg.scale_y_sqrt() +
         gg.ggtitle(disease))
    display(image_png(p, height=600))

<!-- label:widget_ggplot -->

In [ ]:
from ipywidgets import interact
interact(myplot,
         disease=['PNEUMONIA','INFLUENZA','MEASLES'],
         state=['MA','NH','CA'])

---

<!-- label:bokeh -->
<!-- config:split-output -->

In [ ]:
from bokeh.plotting import (figure, show,
                            ColumnDataSource,
                            output_notebook)
##from bokeh.resources import INLINE
output_notebook()

res = res.head(20)
plot = figure(x_range=list(res.rx2('state')))
source = ColumnDataSource({x: tuple(res.rx2(x)) for x in res.colnames})
plot.vbar(x='state',
          bottom=0, top='n',
          width=0.5,
          color='STEELBLUE',
          source=source)

<!-- label:bokeh_show -->

In [ ]:
show(plot)
